# Projet Séminaire Modélisation Statistique

Importations nécessaires:

In [34]:
library(readr)
library(readxl)
library(tidyr)
library(dplyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
#Set directory
setwd("C:/Users/maily/Seminaire-Modelisation-Stat/Data")

### Ouverture et nettoyage des données

In [74]:
data_test <- read.csv("data_test.csv")
data_train <- read.csv("data_train.csv")
benchmark <- read.csv("benchmark.csv")

In [75]:
#Drop NAs
data_test <- data_test %>% drop_na()
data_train <- data_train %>% drop_na()
benchmark <- benchmark %>% drop_na()

In [76]:
data_train

DateTime,Consumption,Temperature,Nebulosity,Humidity
<chr>,<int>,<dbl>,<dbl>,<dbl>
2012-01-01 00:00:00,58315,13.443860,87.50000,87.64912
2012-01-01 03:00:00,52496,13.370175,76.35135,88.26786
2012-01-01 06:00:00,45827,13.743103,78.10638,88.19298
2012-01-01 09:00:00,45730,13.669643,81.27660,87.80000
2012-01-01 12:00:00,50975,15.675439,80.00000,81.67857
2012-01-01 15:00:00,49148,15.442857,83.37209,80.67857
2012-01-01 18:00:00,54425,13.592727,86.32353,85.25455
2012-01-01 21:00:00,56054,12.868421,88.63636,87.61404
2012-01-02 00:00:00,55164,11.869091,87.61290,88.10909


On crée une colonne pour l'année, le mois, le jour, le jour de la semaine et l'heure.

In [77]:
data_train = data_train %>% 
            separate(DateTime, c("Date", "Time"), sep = " ", remove = TRUE) %>% 
            separate(Date, c("Year", "Month", "Day"), sep = "-", remove = FALSE)

In [78]:
data_train$Date = as.Date(data_train$Date, format = "%Y-%m-%d")
data_train["Weekday"] =  weekdays(data_train$Date)
data_train

Date,Year,Month,Day,Time,Consumption,Temperature,Nebulosity,Humidity,Weekday
<date>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>
2012-01-01,2012,01,01,00:00:00,58315,13.443860,87.50000,87.64912,dimanche
2012-01-01,2012,01,01,03:00:00,52496,13.370175,76.35135,88.26786,dimanche
2012-01-01,2012,01,01,06:00:00,45827,13.743103,78.10638,88.19298,dimanche
2012-01-01,2012,01,01,09:00:00,45730,13.669643,81.27660,87.80000,dimanche
2012-01-01,2012,01,01,12:00:00,50975,15.675439,80.00000,81.67857,dimanche
2012-01-01,2012,01,01,15:00:00,49148,15.442857,83.37209,80.67857,dimanche
2012-01-01,2012,01,01,18:00:00,54425,13.592727,86.32353,85.25455,dimanche
2012-01-01,2012,01,01,21:00:00,56054,12.868421,88.63636,87.61404,dimanche
2012-01-02,2012,01,02,00:00:00,55164,11.869091,87.61290,88.10909,lundi


Ajout d'une variable jour ferié.

In [ ]:
jours_feries <- read.csv("jours_feries_metropole.csv")

In [82]:
jours_feries$date = as.Date(jours_feries$date, format = "%Y-%m-%d")
jours_feries = jours_feries %>%
    filter(format(date, format = "%Y") > 2011) %>%
    select (-zone,-annee)

ERROR: Error: Can't subset columns that don't exist.
[31mx[39m Column `zone` doesn't exist.


In [83]:
jours_feries = rename(jours_feries,Date=date)

In [90]:
data_train = left_join(data_train,jours_feries)

Joining, by = "Date"



In [97]:
data_train$nom_jour_ferie[is.character(data_train$nom_jour_ferie) & is.na(data_train$nom_jour_ferie)] = 0
data_train = rename(data_train, jour_ferie = nom_jour_ferie)
data_train

Date,Year,Month,Day,Time,Consumption,Temperature,Nebulosity,Humidity,Weekday,jour_ferie
<date>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2012-01-01,2012,01,01,00:00:00,58315,13.443860,87.50000,87.64912,dimanche,1er janvier
2012-01-01,2012,01,01,03:00:00,52496,13.370175,76.35135,88.26786,dimanche,1er janvier
2012-01-01,2012,01,01,06:00:00,45827,13.743103,78.10638,88.19298,dimanche,1er janvier
2012-01-01,2012,01,01,09:00:00,45730,13.669643,81.27660,87.80000,dimanche,1er janvier
2012-01-01,2012,01,01,12:00:00,50975,15.675439,80.00000,81.67857,dimanche,1er janvier
2012-01-01,2012,01,01,15:00:00,49148,15.442857,83.37209,80.67857,dimanche,1er janvier
2012-01-01,2012,01,01,18:00:00,54425,13.592727,86.32353,85.25455,dimanche,1er janvier
2012-01-01,2012,01,01,21:00:00,56054,12.868421,88.63636,87.61404,dimanche,1er janvier
2012-01-02,2012,01,02,00:00:00,55164,11.869091,87.61290,88.10909,lundi,0
